In [17]:
import sys
sys.path.append("..")
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder
from transformers import AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch import nn
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle
import re

In [18]:
valid_dataset = load_from_disk("/opt/ml/data/train_dataset/validation")

In [19]:
with open('/opt/ml/data/elastic_valid_100.bin','rb') as f:
    elastic_valid = pickle.load(f)
with open('/opt/ml/data/dense_valid_retrieval.bin','rb') as f:
    dense_valid = pickle.load(f)
with open("/opt/ml/data/hybrid_valid_retrieval.bin",'rb') as f:
    hybrid_valid = pickle.load(f)

In [20]:
query = valid_dataset['question']
context = valid_dataset['context']

In [21]:
def preprocess(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\\n', ' ', text) # remove newline character
    text = re.sub(r'\s+', ' ', text) # remove continuous spaces
    text = re.sub(r'#', ' ', text)
    return text

In [27]:

top_k_list = [1,2,3,4,5,6,7,8,9,10]

for top_k in top_k_list:
    tfdif_acc = 0
    dense_acc = 0
    hybrid_acc = 0
    for i in range(len(query)):
        q = query[i]
        ground_truth = preprocess(context[i])
        if ground_truth in elastic_valid[q][:top_k]:
            tfdif_acc += 1
        if ground_truth in dense_valid[q][:top_k]:
            dense_acc += 1
        if ground_truth in hybrid_valid[q][:top_k]:
            hybrid_acc += 1

    print('score_top_k : ', top_k)
    print('elastic ACC : ', tfdif_acc / len(query))
    print('Dense ACC : ', dense_acc / len(query))
    print('Hybrid ACC :',hybrid_acc / len(query))
    print()


score_top_k :  1
elastic ACC :  0.7166666666666667
Dense ACC :  0.2625
Hybrid ACC : 0.45

score_top_k :  2
elastic ACC :  0.7625
Dense ACC :  0.3458333333333333
Hybrid ACC : 0.5791666666666667

score_top_k :  3
elastic ACC :  0.8041666666666667
Dense ACC :  0.38333333333333336
Hybrid ACC : 0.6666666666666666

score_top_k :  4
elastic ACC :  0.8333333333333334
Dense ACC :  0.4375
Hybrid ACC : 0.725

score_top_k :  5
elastic ACC :  0.8583333333333333
Dense ACC :  0.45416666666666666
Hybrid ACC : 0.7458333333333333

score_top_k :  6
elastic ACC :  0.8666666666666667
Dense ACC :  0.4875
Hybrid ACC : 0.7708333333333334

score_top_k :  7
elastic ACC :  0.8791666666666667
Dense ACC :  0.5
Hybrid ACC : 0.8125

score_top_k :  8
elastic ACC :  0.8875
Dense ACC :  0.5041666666666667
Hybrid ACC : 0.8291666666666667

score_top_k :  9
elastic ACC :  0.8958333333333334
Dense ACC :  0.5083333333333333
Hybrid ACC : 0.8375

score_top_k :  10
elastic ACC :  0.9
Dense ACC :  0.5166666666666667
Hybrid ACC 